In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import pandas as pd
import seaborn as sns
sns.set()
import pandas as pd
import gc
gc.collect()
%matplotlib inline
import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')
from tqdm import tqdm

from pyhive import presto
import requests
from requests.auth import HTTPBasicAuth
import pandas as pd
requests.packages.urllib3.disable_warnings()
import matplotlib.pyplot as plt
import datetime as dt
import numpy as np
import gc
gc.collect()
gc.collect()

tqdm.pandas()

/home/ronak/anaconda3/envs/customer_v1/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [15]:
tqdm.pandas()
def download_data(city,date,data_file):
    conn = presto.connect(
        host='10.160.0.62',
        port=8060,
        protocol='http',
        catalog='hive',
        schema='mongodb',
        username='ronak',
        # requests_kwargs=req_kw,
    )
    order_date_query_test = """Select updatedAt,customer,status,orderdate,serviceobj_city,spdfraud_flag
    from hive.mongodb.orders 
    where (orderdate =  '{date}') and (status ='customerCancelled') and (serviceobj_city='{city}')
    """.format(city=city,date=date)
    df_order_date_test = pd.read_sql(order_date_query_test, conn)
    df_order_date_test.to_csv(data_file)

#### Cancelled Rider Issues: (new →  (riderBusy,riderRejected)*  → (customerCancelled))

def get_status_from_slice(each_slice):
    status_index= [i for i, s in enumerate(each_slice.split(',')) if 'status' in s][0]
    first_status=each_slice.split(',')[status_index]
    request_time=float(extract_epoch_timestamp(each_slice.split(',')[status_index+1]))
    return preprocess_status(first_status),request_time
def get_all_new_index(split_list):
    new_string="'status': 'new'"
    all_new_index=[i for i, s in enumerate(split_list) if new_string in s]
    return all_new_index
def get_all_otw_index(split_list):
    new_string="'status': 'onTheWay'"
    all_otw_index=[i for i, s in enumerate(split_list) if new_string in s]
    return all_otw_index

def get_all_riderRejected_index(split_list):
    rider_rejected_string="'status': 'riderRejected'"
    all_rr_index=[i for i, s in enumerate(split_list) if rider_rejected_string in s]
    return all_rr_index

def get_all_cancelled_index(split_list):
    cancelled_String="'status': 'customerCancelled'"
    cancelled_Index=[i for i, s in enumerate(split_list) if cancelled_String in s]
    return cancelled_Index

import re
def extract_epoch_timestamp(a):
    m = re.search('[0-9]{3,}.[0-9]{1}', a)
    if m:
        found = m.group(0)
        return found
def preprocess_status(x):
    return x.replace('\'',"").replace(' ',"").replace('status:',"")   

def get_status_from_slice(each_slice):
    status_index= [i for i, s in enumerate(each_slice.split(',')) if 'status' in s][0]
    first_status=each_slice.split(',')[status_index]
    request_time=float(extract_epoch_timestamp(each_slice.split(',')[status_index+1]))
    return preprocess_status(first_status),request_time

def get_start_stop_indices(split_list):
    new_indices=get_all_new_index(split_list)
    cancelled_indices=get_all_cancelled_index(split_list)
    new_cancelled_indices=list(zip(new_indices,cancelled_indices))
    return new_cancelled_indices
def get_non_none_status_for_range(start,stop,split_list):
    return list(filter(None.__ne__, [get_all_status_for_range(split_list[curr_index]) for curr_index in range(start,stop+1)]))
def get_all_status_for_range(each_slice):
     if 'status' in each_slice:
        status_index= [i for i, s in enumerate(each_slice.split(',')) if 'status' in s][0]
        first_status=each_slice.split(',')[status_index]
        request_time=float(extract_epoch_timestamp(each_slice.split(',')[status_index+1]))
        return preprocess_status(first_status),request_time

def get_if_expected_list(split_list,start_stop):
    status_list=get_non_none_status_for_range(*start_stop,split_list)
    allowed_status=[
        'new',
        'riderBusy',
        'riderRejected',
        'customerCancelled'
    ]
    filtered_list=[x for x in status_list if x[0] in allowed_status]
    return (len(filtered_list)==len(status_list))
def get_count_in_cell(cell):
    split_list= cell.split('{')
    return sum([get_if_expected_list(split_list,start_stop) for start_stop in get_start_stop_indices(split_list)])

def drop_unnamed(df):
    return df.loc[:, ~df.columns.str.contains('^Unnamed')]

def get_cri(data_file):
    orders=pd.read_csv(data_file)
    orders=drop_unnamed(orders)
    orders.head(2)
    updated_at_split=orders['updatedAt'].progress_apply(lambda x: x.strip('[]'))
    count=updated_at_split.progress_apply(get_count_in_cell).sum()
    return count
def execute(city,date):
    data_file='CRI_{city}_{date}.csv'.format(city=city,date=date)
    download_data(city,date,data_file)
    count=get_cri(data_file)
    from pathlib import Path
    Path(data_file+'_'+str(count)).touch()
    print('# CancelledRiderIssues- (new →  (riderBusy,riderRejected)*  → (customerCancelled)) = {0} for {1}'.format(count,data_file))

In [18]:
date_list=['2020-02-03']
city_list=['Kolkata','Delhi']
[execute(city,date) for city in city_list for date in date_list]

100%|██████████| 16719/16719 [00:00<00:00, 30294.65it/s]


# CancelledRiderIssues- (new →  (riderBusy,riderRejected)*  → (customerCancelled)) = 5070 for CRI_Kolkata_2020-02-03.csv


100%|██████████| 11589/11589 [00:00<00:00, 30344.57it/s]

# CancelledRiderIssues- (new →  (riderBusy,riderRejected)*  → (customerCancelled)) = 3461 for CRI_Delhi_2020-02-03.csv


[None, None]

#### Get number of expries